In [1]:
from comet_ml import Experiment
import torch
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import tqdm
import numpy as np
from loguru import logger
import json
import glob

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from torch import nn
import torch.nn.functional as F

In [4]:
logger.remove()
logger.add("log.txt")

1

In [5]:
DEVICE = "cuda:1"

## Load models for vector generation

## Load data

### Read vectors

In [3]:
embeddings = np.load("data/vectors.npy")

In [4]:
indices = np.arange(embeddings.shape[0]//3)

train_idx, test_idx = train_test_split(indices, test_size=0.3, random_state=42)

train_data = [embeddings[train_idx*3], embeddings[train_idx*3+1], embeddings[train_idx*3+2]]
test_data = [embeddings[test_idx*3], embeddings[test_idx*3+1], embeddings[test_idx*3+2]]

print(train_data[0].shape)
print(test_data[0].shape)

(69982, 768)
(29993, 768)


## Measure cosine similarity differences between pos and neg

In [7]:
def calculate_metrics(method, data, name, show_pbar=True):
    orig_shape = data[0].shape[1]
    pos_similarity = []
    neg_similarity = []

    queue = []
    anchors = method(data[0])
    positives = method(data[1])
    negatives = method(data[2])
   
    for i in tqdm.trange(len(anchors), disable=not show_pbar):
        query, pos, neg = anchors[i], positives[i], negatives[i]
        pos_similarity.append(cosine_similarity([query], [pos])[0][0])
        neg_similarity.append(cosine_similarity([query], [neg])[0][0])

    pos_similarity = np.array(pos_similarity)
    neg_similarity = np.array(neg_similarity)
    
    ratio = np.sum(pos_similarity>neg_similarity)/len(pos_similarity)
    mean_diff = pos_similarity.mean()-neg_similarity.mean()
    mean_pos_sim = pos_similarity.mean()
    
    reduction_rate = orig_shape/len(anchors[0])
    return pd.DataFrame.from_dict([{
                        "ratio": ratio,
                        "mean_pos_sim": mean_pos_sim,
                        "mean_diff": mean_diff,
                        "method": name,
                        "reduction_rate": reduction_rate
                    }])

## Benchmarks

In [8]:
metrics = []

In [9]:
def validate_method(get_method, test_method):
    res = []
    for input_dim in tqdm.notebook.tqdm([8, 16, 32, 64, 128, 768]):
        for output_dim in tqdm.notebook.tqdm([2, 4, 8, 16, 32, 64, 128]):
            if input_dim <= output_dim:
                continue

            model = get_method(input_dim, output_dim)
            res.append(calculate_metrics(test_method(model, input_dim), test_data, f"PCA ({input_dim}x{output_dim})", show_pbar=False))
    return res

def aggregate_metrics(directory="results"):
    dframes = []
    for method in glob.glob(f"{directory}/*.csv"):
        if "total.csv" not in method:
            dframes.append(pd.read_csv(method))
    aggregated = pd.concat(dframes)
    aggregated = aggregated.sort_values(["reduction_rate", "ratio"], ascending=[False, True])
    aggregated.to_csv(f"{directory}/total.csv")
    return aggregated

def save_metrics(metrics, fname):
    cmetrics = pd.concat(metrics)

    max_ratio = cmetrics.groupby("reduction_rate").max()["ratio"]
    cmetrics = cmetrics[cmetrics.apply(lambda x: x["ratio"]==max_ratio[x["reduction_rate"]], axis=1)]
    cmetrics = cmetrics.sort_values("reduction_rate", ascending=False)
    cmetrics.to_csv(f"results/{fname}.csv")

    return cmetrics

### Baseline

In [44]:
metrics.append(calculate_metrics(lambda x: x, test_data, "CLS vector"))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 29993/29993 [00:09<00:00, 3279.73it/s]


In [ ]:
# lda_metrics = []
# embeddings = []

# train_data = data.sample(1000, random_state=42)

# for i in tqdm.trange(0, train_data.shape[0], 32):
#     embeddings.extend(embed(

In [40]:
dim = 64

# for idx in range(0, train.shape

In [42]:
for

,query,pos,neg
15773,what does dsm stand for,"In psychology and psychiatry, DSM stands for t...","military mk, what does mark stand for in milit..."
35290,what is social competence in early childhood,Social competence in early childhood is as a s...,The Early Years Learning Framework (EYLF) is d...
358404,stonefield construction phone number,A: The phone number for Stonefield Constructio...,The Building Official reviews applications for...
381806,what book did adam smith write,"Adam Smith FRSA was a Scottish economist, phil...",Was Adam Smith laissez-faire? What did it mean...
244560,how long can you take naltrexone for,You and your doctor have to decide this. Most ...,"Naloxone, also known as Narcan among other nam..."
...,...,...,...
178265,what happens in the first stage of seed germin...,Stage One - Hydration and Metabolism. In the f...,Germination is the process in which the seeds ...
168368,aberdeen airport code,Aberdeen International Airport (Scottish Gaeli...,Welcome to USZip.com. USZip.com USZip.com is a...
314866,how old till baby can sleep on stomach,You may find yourself wondering if your baby c...,Third Month Baby Milestones: Sleep. Your 3-mon...
472372,what are the sisters last names in in the time...,In the Time of the Butterflies Characters. BÃ©...,The nine brightest stars of the Pleiades are n...


### Averaging

In [379]:
def average_vec(dim):
    def _inner(embeddings):
        x = embeddings.reshape(-1, dim)
        x = x.mean(axis=1).reshape(-1, 1)
        x = x.reshape(len(embeddings), -1)
        x = x/np.sqrt(np.sum(x**2))
        return x
    return _inner

In [380]:
for rate in [2, 4, 8, 12, 16, 24, 32, 48, 64, 96, 128, 192, 384]:
    method = average_vec(rate)
    metrics.append(calculate_metrics(method, test_data, f"Average parts ({rate})"))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29993/29993 [00:11<00:00, 2681.76it/s]


In [381]:
res = save_metrics(metrics, "averaging")
aggregate_metrics()

,Unnamed: 0,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0,0.143567,0.732938,0.137099,PCA (768x1),768.0
0,0,0.550879,0.668035,0.040766,UMAP (768x2),384.0
0,0,0.584403,0.562329,0.183465,Average parts (384),384.0
1,0,0.593138,0.595463,0.134497,PCA (384x1),384.0
0,0,0.610883,0.810334,0.124641,Whitening (768x2),384.0
...,...,...,...,...,...,...
4,0,0.958501,0.987163,0.010953,Autoencoder,2.0
13,0,0.960758,0.750421,0.205391,Siamese network (96x48),2.0
12,0,0.964125,0.733570,0.229238,Average parts (2),2.0
12,0,0.964714,0.738850,0.224645,Whitening (128x64),2.0


### Summing

In [391]:
metrics = []
def average_vec(dim):
    def _inner(embeddings):
        embeddings = embeddings/embeddings.sum(axis=1)[:, None]
        x = embeddings.reshape(-1, dim)
        x = x.sum(axis=1).reshape(-1, 1)
        x = x.reshape(len(embeddings), -1)
        # x = x/np.sqrt(np.sum(x**2))
        return x
    return _inner

In [392]:
for rate in [2, 4, 8, 12, 16, 24, 32, 48, 64, 96, 128, 192, 384]:
    method = average_vec(rate)
    metrics.append(calculate_metrics(method, test_data, f"Sum parts ({rate})"))

 69%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                  | 20641/29993 [00:07<00:03, 2601.45it/s]


KeyboardInterrupt: 

In [ ]:
res = save_metrics(metrics, "sums")
aggregate_metrics()

### Generic autoencoder

In [399]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.encoder = nn.Sequential(
                    nn.Linear(input_dim, 32),
                    nn.ReLU(),
                    nn.Linear(32, output_dim),
            )
        self.decoder = nn.Sequential(
                    nn.Linear(output_dim, 32),
                    nn.ReLU(),
                    nn.Linear(32, input_dim)
            )
    
    def forward(self, x):
        x = self.encoder(x)
        x = F.normalize(x)
        x = self.decoder(x)
        x = F.normalize(x)
        return x
    
    def encode(self, x):
        x = self.encoder(x)
        x = F.normalize(x)
        return x
    
def test_autoenc(model, dim):
    def _inner(embeddings):
        x = embeddings.reshape(-1, dim)
        x = torch.from_numpy(x).to(torch.float32)
        x = model.encode(x).cpu().detach().numpy()
        x = x.reshape(len(embeddings), -1)
        return x
    return _inner

In [394]:
data = torch.tensor(train_data).reshape(-1, 768)
train_loader = torch.utils.data.DataLoader(data.to(torch.float32), batch_size=32, shuffle=True)

batch_size = 32

def train_autoenc(model, dim, criterion, epochs=1, params={}):
    optim = torch.optim.Adam(model.parameters(), 3e-4)
    
    experiment = Experiment(
    api_key="sQnInDpH3eOHuFvstNS3jG5jb",
    project_name="vector-compression",
    workspace="wwydmanski",
    display_summary_level=0,
    auto_metric_logging=False
    )
    experiment.add_tag("autoencoder")
    experiment.log_parameters(params)
    
    for epoch in range(1, epochs+1):
        with tqdm.notebook.tqdm(train_loader) as t:
            total_loss = []

            for step, batch in enumerate(t):
                x = batch.reshape(-1, dim)
                batch_loss = []
                for idx in range(0, len(x), batch_size):
                    minibatch = x[idx:idx+batch_size]

                    optim.zero_grad()

                    y_hat = model(minibatch)
                    loss = criterion(minibatch, y_hat)
                    loss.backward()
                    optim.step()

                    batch_loss.append(loss.item())

                experiment.log_metric("batch_loss_", np.mean(batch_loss), step=step, epoch=epoch)

            experiment.log_metric("loss_", np.mean(total_loss), step=step, epoch=epoch)

        res = calculate_metrics(test_autoenc(model, dim), train_data, f"Autoencoder")
        print(res)
        experiment.log_metric("Train accuracy", res["ratio"].values[0], step=epoch*step)
        experiment.log_metric("Train mean positive similarity", res["mean_pos_sim"].values[0], step=epoch*step)
        experiment.log_metric("Train mean difference", res["mean_diff"].values[0], step=epoch*step)
        
    res = calculate_metrics(test_autoenc(model, dim), test_data, f"Autoencoder")
    print(res)
    experiment.log_metric("Test accuracy", res["ratio"].values[0], step=epoch*step)
    experiment.log_metric("Test mean positive similarity", res["mean_pos_sim"].values[0], step=epoch*step)
    experiment.log_metric("Test mean difference", res["mean_diff"].values[0], step=epoch*step)
        
    experiment.log_parameter("Reduction rate", res["reduction_rate"].values[0])
    experiment.end()

In [400]:
metrics = []

In [ ]:
hinge_margin = torch.tensor(0)
gamma = 1
def var_loss(x):
    var = torch.sqrt(x.var(0)+1e-6)
    loss = torch.maximum(hinge_margin, gamma-var).mean()
    return loss


dim=32
output_dim = 8
lr = 3e-4

for input_dim in tqdm.notebook.tqdm([8, 12, 16, 32, 64, 96, 128, 192, 384, 768]):
    for output_dim in tqdm.notebook.tqdm([1, 2, 4, 6, 8, 12, 16, 32, 64, 96, 128]):
        if input_dim <= output_dim:
            continue
        if input_dim%output_dim != 0:
            continue
            
        params = {"Input dim": input_dim, "Output dim": output_dim, "LR": lr}

        model = Autoencoder(input_dim, output_dim)

        optim = torch.optim.Adam(model.parameters(), lr)
        criterion_ = nn.CosineSimilarity()
        criterion = lambda x, y: -criterion_(x, y).mean()

        train_autoenc(model, input_dim, criterion, epochs=4, params=params)
        metrics.append(calculate_metrics(test_autoenc(model, input_dim), test_data, f"Autoencoder ({input_dim}x{output_dim})"))
        logger.info(metrics[-1])

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wwydmanski/vector-compression/8b324d9596864af1b56f456f2ed06b64



  0%|          | 0/6561 [00:00<?, ?it/s]

In [390]:
metrics

[]

In [389]:
cmetrics = pd.concat(metrics)

max_ratio = cmetrics.groupby("reduction_rate").max()["ratio"]
cmetrics = cmetrics[cmetrics.apply(lambda x: x["ratio"]==max_ratio[x["reduction_rate"]], axis=1)]
cmetrics = cmetrics.sort_values("reduction_rate", ascending=False)
# cmetrics.to_csv("results/autoencoder.csv")

cmetrics

ValueError: No objects to concatenate

### PCA

In [353]:
metrics = []

In [354]:
from sklearn.decomposition import PCA

In [357]:
def test_pca(model, input_dim):
    def _inner(embeddings):
        x = embeddings.reshape(-1, input_dim)
        x = model.transform(x)
        x = x.reshape(len(embeddings), -1)
        x = x/np.sqrt(np.sum(x**2))
        return x
    return _inner

def get_pca(input_dim, output_dim):
    model = PCA(output_dim)
    # data = train_data[0].reshape(-1, input_dim)
    data = np.concatenate(train_data).reshape(-1, input_dim)
    model.fit(data)
    return model

In [358]:
input_dim = 64
output_dim = 8

model = get_pca(input_dim, output_dim)
calculate_metrics(test_pca(model, input_dim), test_data, f"PCA ({input_dim}x{output_dim})")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29993/29993 [00:10<00:00, 2855.68it/s]


,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0.947754,0.738982,0.224128,PCA (64x8),8.0


In [362]:
for input_dim in tqdm.notebook.tqdm([8, 12, 16, 32, 64, 96, 128, 192, 384, 768]):
    for output_dim in tqdm.notebook.tqdm([1, 2, 4, 6, 8, 12, 16, 32, 64, 96, 128]):
        if input_dim <= output_dim:
            continue
        if input_dim%output_dim != 0:
            continue
            
        model = get_pca(input_dim, output_dim)
        metrics.append(calculate_metrics(test_pca(model, input_dim), test_data, f"PCA ({input_dim}x{output_dim})", show_pbar=False))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

In [364]:
save_metrics(metrics, "PCA")
aggregate_metrics()

,Unnamed: 0,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0,0.143567,0.732938,0.137099,PCA (768x1),768.0
0,0,0.550879,0.668035,0.040766,UMAP (768x2),384.0
0,0,0.584403,0.562329,0.183465,Average parts (384),384.0
1,0,0.593138,0.595463,0.134497,PCA (384x1),384.0
0,0,0.610883,0.810334,0.124641,Whitening (768x2),384.0
...,...,...,...,...,...,...
12,0,0.934006,0.676887,0.184324,UMAP (32x16),2.0
4,0,0.958501,0.987163,0.010953,Autoencoder,2.0
13,0,0.960758,0.750421,0.205391,Siamese network (96x48),2.0
12,0,0.964714,0.738850,0.224645,Whitening (128x64),2.0


#### Save the models

In [371]:
import pickle

In [370]:
model = get_pca(64, 4)

In [372]:
with open("models/PCA_64x4.pickle", "wb") as f:
    pickle.dump(model, f)

### UMAP

In [365]:
metrics = []

In [366]:
from cuml.manifold import UMAP

In [367]:
def get_umap(input_dim, output_dim):
    model = UMAP(n_components=output_dim)
    data = train_data[0][:10000].reshape(-1, input_dim)
    model.fit(data)
    return model

In [368]:
model = get_umap(128, 16)

In [369]:
for input_dim in tqdm.notebook.tqdm([8, 12, 16, 32, 64, 96, 128, 192, 384, 768]):
    for output_dim in tqdm.notebook.tqdm([1, 2, 4, 6, 8, 12, 16, 32, 64, 96, 128]):
        if input_dim <= output_dim:
            continue
        if input_dim%output_dim != 0:
            continue
            
        model = get_umap(input_dim, output_dim)
        metrics.append(calculate_metrics(test_pca(model, input_dim), test_data, f"UMAP ({input_dim}x{output_dim})", show_pbar=False))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [35]:
cmetrics = pd.concat(metrics)

max_ratio = cmetrics.groupby("reduction_rate").max()["ratio"]
cmetrics = cmetrics[cmetrics.apply(lambda x: x["ratio"]==max_ratio[x["reduction_rate"]], axis=1)]
cmetrics = cmetrics.sort_values("reduction_rate", ascending=False)
cmetrics.to_csv("results/UMAP.csv")

cmetrics

,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0.550879,0.668035,0.040766,UMAP (768x2),384.0
0,0.632088,0.911761,0.057221,UMAP (768x4),192.0
0,0.651104,0.957159,0.033607,UMAP (768x8),96.0
0,0.544500,0.846501,0.028204,UMAP (128x2),64.0
0,0.653393,0.968231,0.024588,UMAP (768x16),48.0
0,0.644124,0.954982,0.030882,UMAP (128x4),32.0
0,0.650470,0.975806,0.017704,UMAP (768x32),24.0
0,0.673565,0.965916,0.033695,UMAP (128x8),16.0
0,0.614295,0.824949,0.034135,UMAP (768x64),12.0
0,0.885116,0.629663,0.168973,UMAP (32x4),8.0


### LocallyLinearEmbedding

In [8]:
from sklearn.manifold import LocallyLinearEmbedding

In [63]:
batch_size = 512
def test_lle(model, input_dim):
    def _inner(embeddings):
        res = []
        x = embeddings.reshape(-1, input_dim)
        
        for i in range(0, len(x), batch_size):
            res.append(model.transform(x[i:i+batch_size]))

        x = np.concatenate(res)
        x = x.reshape(len(embeddings), -1)
        return x
    return _inner

def get_lle(input_dim, output_dim):
    model = LocallyLinearEmbedding(n_components=output_dim, n_jobs=20)
    data = train_data[0].reshape(-1, input_dim)[:5000]
    model.fit(data)
    return model

In [64]:
metrics = []

In [65]:
for input_dim in tqdm.notebook.tqdm([8, 16, 32, 64, 128, 768][::-1]):
    for output_dim in tqdm.notebook.tqdm([2, 4, 8, 16, 32, 64, 128]):
        if input_dim <= output_dim:
            continue
            
        model = get_lle(input_dim, output_dim)
        metrics.append(calculate_metrics(test_lle(model, input_dim), test_data, f"LocallyLinearEmbedding ({input_dim}x{output_dim})", show_pbar=False))

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [66]:
cmetrics = pd.concat(metrics)

max_ratio = cmetrics.groupby("reduction_rate").max()["ratio"]
cmetrics = cmetrics[cmetrics.apply(lambda x: x["ratio"]==max_ratio[x["reduction_rate"]], axis=1)]
cmetrics = cmetrics.sort_values("reduction_rate", ascending=False)
# cmetrics.to_csv("results/LocallyLinear.csv")

cmetrics

,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0.624612,0.796312,0.109099,LocallyLinearEmbedding (768x2),384.0
0,0.658020,0.819372,0.113057,LocallyLinearEmbedding (768x4),192.0
0,0.673090,0.826621,0.110064,LocallyLinearEmbedding (768x8),96.0
0,0.622545,0.653948,0.111262,LocallyLinearEmbedding (128x2),64.0
0,0.688461,0.812073,0.126374,LocallyLinearEmbedding (768x16),48.0
0,0.737639,0.451451,0.179626,LocallyLinearEmbedding (64x2),32.0
0,0.707932,0.776378,0.152379,LocallyLinearEmbedding (768x32),24.0
0,0.803888,0.439338,0.183319,LocallyLinearEmbedding (32x2),16.0
0,0.720935,0.741271,0.172130,LocallyLinearEmbedding (768x64),12.0
0,0.852099,0.424947,0.181779,LocallyLinearEmbedding (32x4),8.0


### Whitening Sentence Representations 

In [5]:
def compute_kernel_bias(input_dim):
    """
    y = (x + bias).dot(kernel)
    """
    vecs = train_data[0].reshape(-1, input_dim)
    # vecs = np.concatenate(vecs, axis=0)
    mu = vecs.mean(axis=0, keepdims=True)
    cov = np.cov(vecs.T)
    u, s, vh = np.linalg.svd(cov)
    W = np.dot(u, np.diag(1/np.sqrt(s)))
    return W, -mu

def transform_and_normalize(vecs, kernel, bias):
    if not (kernel is None or bias is None):
        vecs = (vecs + bias).dot(kernel)
    normalized=vecs / (vecs**2).sum(axis=1, keepdims=True)**0.5
    return normalized

def test_whitening(kernel, bias,input_dim):
    def _inner(embeddings):
        x = embeddings.reshape(-1, input_dim)
        x = transform_and_normalize(x,kernel=kernel,bias=bias)
        x = x.reshape(len(embeddings), -1)
        return x
    return _inner

In [6]:
metrics = []

In [7]:
for input_dim in [32, 64, 128, 768]:
    for output_dim in [2, 4, 8, 16, 32, 64, 128]:
        if input_dim <= output_dim:
            continue
        kernel, bias = compute_kernel_bias(input_dim)
        kernel = kernel[:, :output_dim]
        metrics.append(calculate_metrics(test_whitening(kernel,bias,input_dim), test_data, f"Whitening ({input_dim}x{output_dim})"))

 93%|████████████████████████████████████████████████████████████████████████████████████████████████▉       | 27966/29993 [00:09<00:00, 3097.27it/s]


KeyboardInterrupt: 

In [55]:
cmetrics = pd.concat(metrics)

max_ratio = cmetrics.groupby("reduction_rate").max()["ratio"]
cmetrics = cmetrics[cmetrics.apply(lambda x: x["ratio"]==max_ratio[x["reduction_rate"]], axis=1)]
cmetrics = cmetrics.sort_values("reduction_rate", ascending=False)
cmetrics.to_csv("results/Whitening.csv")
cmetrics

,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0.610883,0.810334,0.124641,Whitening (768x2),384.0
0,0.683523,0.840056,0.128928,Whitening (768x4),192.0
0,0.751950,0.838032,0.142059,Whitening (768x8),96.0
0,0.748850,0.652865,0.199488,Whitening (128x2),64.0
0,0.833070,0.826492,0.163008,Whitening (768x16),48.0
0,0.839594,0.703681,0.209091,Whitening (128x4),32.0
0,0.902454,0.806293,0.190766,Whitening (768x32),24.0
0,0.902409,0.724704,0.213697,Whitening (128x8),16.0
0,0.937863,0.793159,0.210244,Whitening (768x64),12.0
0,0.938040,0.735265,0.217495,Whitening (128x16),8.0


### Siamese network

In [321]:
data = torch.tensor(train_data).permute(1,0,2)
triplet_loader = torch.utils.data.DataLoader(data.to(torch.float32), batch_size=32, shuffle=True)


class EmbeddingHead(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.encoder = nn.Sequential(
                    nn.Linear(input_dim, output_dim),
            )
    
    def forward(self, x):
        x = self.encoder(x)
        x = F.normalize(x)
        return x

def get_embedding_model(input_dim, hidden_dim, output_dim):
    encoder = EmbeddingHead(input_dim, hidden_dim, output_dim)
    return encoder

def test_embedding(model, dim):
    def _inner(embeddings):
        x = embeddings.reshape(-1, dim)
        x = torch.from_numpy(x).to(torch.float32)
        x = model(x).cpu().detach().numpy()
        x = x.reshape(len(embeddings), -1)
        return x
    return _inner

In [326]:
batch_size = 32

def train_siamese(model, dim, epochs=1, params=None):
    optim = torch.optim.Adam(model.parameters(), params["LR"])
    
    experiment = Experiment(
    api_key="sQnInDpH3eOHuFvstNS3jG5jb",
    project_name="vector-compression",
    workspace="wwydmanski",
    display_summary_level=0,
    auto_metric_logging=False
    )
    experiment.add_tag("siamese-network")
    experiment.log_parameters(params)
    
    for epoch in tqdm.notebook.trange(1, epochs+1):
        total_loss = []
        model.train()
        for step, sample in enumerate(triplet_loader):
            sample = sample.reshape(-1, 3, dim)
            anchor, pos, neg = sample[:, 0], sample[:, 1], sample[:, 2]

            batch_loss = []
            for idx in range(0, len(anchor), batch_size):
                anchor_ = anchor[idx:idx+batch_size]
                pos_ = pos[idx:idx+batch_size]
                neg_ = neg[idx:idx+batch_size]

                optim.zero_grad()

                anchor_emb = model(anchor_)
                pos_emb = model(pos_)
                neg_emb = model(neg_)

                # loss = (1-(anchor_emb*pos_emb).sum(axis=1))**2 
                # loss += (anchor_emb*neg_emb).sum(axis=1)**2 
                # loss += (3-torch.norm(anchor_emb, dim=1)-torch.norm(pos_emb, dim=1)-torch.norm(neg_emb, dim=1))**2
                # loss = loss.mean()
                loss = F.triplet_margin_with_distance_loss(anchor_emb, pos_emb, neg_emb, distance_function=lambda a,b: 1-F.cosine_similarity(a, b), margin=0.2)*10
                loss += ((1-torch.norm(anchor_emb, dim=1))**2).mean()
                loss += ((1-torch.norm(pos_emb, dim=1))**2).mean()
                loss += ((1-torch.norm(neg_emb, dim=1))**2).mean()

                loss.backward()
                optim.step()

                batch_loss.append(loss.item())

            experiment.log_metric("batch_loss_", np.mean(batch_loss), step=step, epoch=epoch)

            experiment.log_metric("loss_", np.mean(total_loss), step=step, epoch=epoch)
        
#         model.eval()
#         res = calculate_metrics(test_embedding(model, dim), train_data, f"Siamese network", False)
#         logger.info(res)
#         experiment.log_metric("Train accuracy", res["ratio"].values[0], step=epoch*step)
#         experiment.log_metric("Train mean positive similarity", res["mean_pos_sim"].values[0], step=epoch*step)
#         experiment.log_metric("Train mean difference", res["mean_diff"].values[0], step=epoch*step)
        
#         res = calculate_metrics(test_embedding(model, dim), test_data, f"Siamese network", False)
#         logger.info(res)
#         experiment.log_metric("Test accuracy", res["ratio"].values[0], step=epoch*step)
#         experiment.log_metric("Test mean positive similarity", res["mean_pos_sim"].values[0], step=epoch*step)
#         experiment.log_metric("Test mean difference", res["mean_diff"].values[0], step=epoch*step)
        
    experiment.log_parameter("Reduction rate", res["reduction_rate"].values[0])
    experiment.end()

In [286]:
metrics = []

In [278]:
input_dim = 64
output_dim = 8
lr = 3e-4
hidden_dim = 64

params = {"Input dim": input_dim, "Output dim": output_dim, "LR": lr, "Hidden dim": hidden_dim}
model = get_embedding_model(input_dim, hidden_dim, output_dim)

train_siamese(model, input_dim, epochs=10, params=params)
metrics.append(calculate_metrics(test_embedding(model, input_dim), test_data, f"Siamese network ({input_dim}x{output_dim})", False))
logger.info(metrics[-1])

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wwydmanski/vector-compression/fb4a3ac88a5443038432031bff63dc14



  0%|          | 0/313 [00:00<?, ?it/s]

/home/wwydmanski/.conda/envs/img_processing/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/wwydmanski/.conda/envs/img_processing/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


  0%|          | 0/313 [00:00<?, ?it/s]

/home/wwydmanski/.conda/envs/img_processing/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/wwydmanski/.conda/envs/img_processing/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


  0%|          | 0/313 [00:00<?, ?it/s]

/home/wwydmanski/.conda/envs/img_processing/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/wwydmanski/.conda/envs/img_processing/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


  0%|          | 0/313 [00:00<?, ?it/s]

/home/wwydmanski/.conda/envs/img_processing/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/wwydmanski/.conda/envs/img_processing/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 

In [348]:
dim=32
output_dim = 8
lr = 3e-4
input_dim = 768

for input_dim in [192]:
    for output_dim in tqdm.notebook.tqdm([2, 3, 4, 8, 16, 32]):
        if input_dim <= output_dim:
            continue
        if input_dim%output_dim != 0:
            continue
            
        params = {"Input dim": input_dim, "Output dim": output_dim, "LR": lr}
        logger.info(params)

        model = get_embedding_model(input_dim, hidden_dim, output_dim)

        train_siamese(model, input_dim, epochs=10, params=params)
        metrics.append(calculate_metrics(test_embedding(model, input_dim), test_data, f"Siamese network ({input_dim}x{output_dim})", False))
        logger.info(metrics[-1])

  0%|          | 0/3 [00:00<?, ?it/s]

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wwydmanski/vector-compression/b270b8e8409241dabd8e7c0f4d324222



  0%|          | 0/10 [00:00<?, ?it/s]

COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading 1 metrics, params and output messages
COMET INFO: Waiting for completion of the file uploads (may take several seconds)
COMET INFO: The Python SDK has 10800 seconds to finish before aborting...
COMET INFO: All files uploaded, waiting for confirmation they have been all received
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wwydmanski/vector-compression/b7988cc4a99a4eb5a728b4674f20a4e4



  0%|          | 0/10 [00:00<?, ?it/s]

COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading 1 metrics, params and output messages
COMET INFO: Waiting for completion of the file uploads (may take several seconds)
COMET INFO: The Python SDK has 10800 seconds to finish before aborting...
COMET INFO: All files uploaded, waiting for confirmation they have been all received
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wwydmanski/vector-compression/38aa14e5090e49819dff6e757b3a5bc0



  0%|          | 0/10 [00:00<?, ?it/s]

COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading 1 metrics, params and output messages


In [349]:
metrics[-4:]

[      ratio  mean_pos_sim  mean_diff                  method  reduction_rate
 0  0.930217      0.954959   0.036853  Siamese network (32x4)             8.0,
       ratio  mean_pos_sim  mean_diff                  method  reduction_rate
 0  0.525889      0.705087   0.106453  Siamese network (64x1)            64.0,
       ratio  mean_pos_sim  mean_diff                  method  reduction_rate
 0  0.777615      0.951765   0.032318  Siamese network (64x2)            32.0,
       ratio  mean_pos_sim  mean_diff                  method  reduction_rate
 0  0.892842      0.932043   0.054257  Siamese network (64x4)            16.0]

In [350]:
save_metrics(metrics, "siamese2")

,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0.664555,0.909573,0.148700,Siamese network (768x2) [hidden: 128],384.0
0,0.756276,0.836421,0.212844,Siamese network (768x4) [hidden: 64],192.0
0,0.822292,0.771496,0.251995,Siamese network (768x8) [hidden: 128],96.0
0,0.725103,0.962201,0.019704,Siamese network (128x2) [hidden: 64],64.0
0,0.871403,0.728268,0.269258,Siamese network (768x16) [hidden: 128],48.0
0,0.827526,0.932969,0.044065,Siamese network (128x4) [hidden: 64],32.0
0,0.895242,0.714993,0.267751,Siamese network (768x32) [hidden: 128],24.0
0,0.899910,0.900698,0.073726,Siamese network (128x8) [hidden: 128],16.0
0,0.916914,0.863857,0.104821,Siamese network (96x8),12.0
0,0.935018,0.892269,0.088211,Siamese network (64x8) [hidden: 64],8.0


In [303]:
cmetrics = pd.concat(metrics)
cmetrics[cmetrics["reduction_rate"]==2]

,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0.957357,0.830561,0.114643,Siamese network (32x16) [hidden: 64],2.0
0,0.955323,0.798326,0.150121,Siamese network (64x32) [hidden: 64],2.0
0,0.946054,0.811501,0.148327,Siamese network (128x64) [hidden: 64],2.0
0,0.955956,0.812630,0.120257,Siamese network (32x16) [hidden: 128],2.0
0,0.957357,0.781193,0.154783,Siamese network (64x32) [hidden: 128],2.0
0,0.951122,0.817491,0.142804,Siamese network (128x64) [hidden: 128],2.0
0,0.925616,0.692671,0.265935,Siamese network (768x384) [hidden: 64],2.0
0,0.894775,0.671352,0.280919,Siamese network (768x384) [hidden: 64],2.0


In [352]:
save_metrics(metrics, "siamese")
aggregate_metrics()

,Unnamed: 0,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0,0.550879,0.668035,0.040766,UMAP (768x2),384.0
0,0,0.552829,0.385381,0.106327,PCA (768x2),384.0
0,0,0.584403,0.562329,0.183465,Average parts (384),384.0
0,0,0.610883,0.810334,0.124641,Whitening (768x2),384.0
0,0,0.625553,0.839529,0.093303,LocallyLinearEmbedding (768x2),384.0
...,...,...,...,...,...,...
12,0,0.934006,0.676887,0.184324,UMAP (32x16),2.0
4,0,0.958501,0.987163,0.010953,Autoencoder,2.0
13,0,0.960758,0.750421,0.205391,Siamese network (96x48),2.0
12,0,0.964714,0.738850,0.224645,Whitening (128x64),2.0


## Similarity embedding framework

In [5]:
import sef_dr

# SEF PCA

In [6]:
def test_sef(model, input_dim):
    def _inner(embeddings):
        x = embeddings.reshape(-1, input_dim)
        x = model.transform(x)
        x = x.reshape(len(embeddings), -1)
        return x
    return _inner

def get_pca(input_dim, output_dim):
    model = PCA(output_dim)
    data = train_data[0].reshape(-1, input_dim)
    model.fit(data)
    return model

In [ ]:
metrics=[]
for input_dim in [64, 128, 768]:
    for target_dim in [32, 64, 128]:
        for output_dim in [32, 64, 128]:
            if input_dim <= output_dim or output_dim >= target_dim or target_dim>=input_dim:
                continue
            print(f"input {input_dim} output {output_dim} target {target_dim}")
            
            data = torch.tensor(train_data[0]).reshape(-1, input_dim)
            model = get_pca(input_dim, target_dim)
            model.fit(data)
            target_data = np.float32(model.transform(data))
            print("model built")
            proj = sef_dr.LinearSEF(input_dimensionality=input_dim, output_dimensionality=output_dim)
            print("-")
            loss = proj.fit(data=data, target_data=target_data, target='copy', epochs=50, batch_size=128, verbose=True, learning_rate=0.001, regularizer_weight=0.001)
            print("trianed")
            metrics.append(calculate_metrics(test_sef(proj,input_dim), test_data, f"SEF PCA ({input_dim}x{output_dim}|{target_dim})"))

input 128 output 32 target 64
model built
-


In [ ]:
cmetrics = pd.concat(metrics)

max_ratio = cmetrics.groupby("reduction_rate").max()["ratio"]
cmetrics = cmetrics[cmetrics.apply(lambda x: x["ratio"]==max_ratio[x["reduction_rate"]], axis=1)]
cmetrics = cmetrics.sort_values("reduction_rate", ascending=False)
cmetrics.to_csv("results/SEF_PCA.csv")
cmetrics

# SEF Whitening


In [ ]:
metrics=[]
for input_dim in [64, 128, 768]:
    for target_dim in [32, 64, 128]:
        for output_dim in [32, 64, 128]:
            if input_dim <= output_dim or output_dim >= target_dim or target_dim>=input_dim:
                continue
            print(f"input {input_dim} output {output_dim} target {target_dim}")
            data=train_data[0].reshape(-1, input_dim)
            # data = torch.tensor(train_data[0]).reshape(-1, input_dim)
            kernel, bias = compute_kernel_bias(input_dim)
            kernel = kernel[:, :target_dim]
            target_data = transform_and_normalize(data,kernel=kernel,bias=bias)
            
            print("model built")
            proj = sef_dr.LinearSEF(input_dimensionality=input_dim, output_dimensionality=output_dim)
            loss = proj.fit(data=data, target_data=target_data, target='copy', epochs=50, batch_size=128, verbose=True, learning_rate=0.001, regularizer_weight=0.001)
            print("trianed")
            metrics.append(calculate_metrics(test_sef(proj,input_dim), test_data, f"SEF Whitening({input_dim}x{output_dim}|{target_dim})"))

In [ ]:
cmetrics = pd.concat(metrics)

max_ratio = cmetrics.groupby("reduction_rate").max()["ratio"]
cmetrics = cmetrics[cmetrics.apply(lambda x: x["ratio"]==max_ratio[x["reduction_rate"]], axis=1)]
cmetrics = cmetrics.sort_values("reduction_rate", ascending=False)
cmetrics.to_csv("results/SEF_Whitening.csv")


In [ ]:
cmetrics

# SEF Averaging

In [ ]:
metrics=[]
for input_dim in [64, 128, 768]:
    for target_rate in [4, 8, 16, 32, 64, 128]:
        for output_dim in [32, 64, 128]:
            target_dim=input_dim/target_rate
            if input_dim <= output_dim or output_dim >= target_dim or target_dim>=input_dim:
                continue
            print(f"input {input_dim} output {output_dim} target {target_rate}")
            target_data = train_data[0].reshape(-1, target_rate)
            target_data.mean(axis=1).reshape(-1, 1)
            target_data = data.reshape(len(train_data[0]), -1)
            print(target_data.shape)
            print("model built")
            data=train_data[0].reshape(-1, input_dim)
            proj = sef_dr.LinearSEF(input_dimensionality=input_dim, output_dimensionality=output_dim)
            loss = proj.fit(data=data, target_data=target_data, target='copy', epochs=50, batch_size=128, verbose=True, learning_rate=0.001, regularizer_weight=0.001)
            print("trianed")
            metrics.append(calculate_metrics(test_sef(proj,input_dim), test_data, f"SEF Averaging({input_dim}x{output_dim}|{target_rate})"))

In [ ]:
cmetrics = pd.concat(metrics)

max_ratio = cmetrics.groupby("reduction_rate").max()["ratio"]
cmetrics = cmetrics[cmetrics.apply(lambda x: x["ratio"]==max_ratio[x["reduction_rate"]], axis=1)]
cmetrics = cmetrics.sort_values("reduction_rate", ascending=False)
cmetrics.to_csv("results/SEF_Averaging.csv")
cmetrics